# Adventures in P-Typing
### Using Natural Language Processing and Data Science to Explore the Myers-Briggs Personality Test
***

The Myers-Brigs Type Indicator (MBTI) is a self-assessment questionnaire which helps people gain insights about how they work and learn. The test was developed by the mother-daughter partnership of Katharine Briggs and Isabel Briggs-Myers and is based on the typological theory of Carl Jung. The length of the test can range from 93 to 222 items that serve to identify a participant's personality type based on four key preferences in how they interract and deal with the world around them. 

These preferances are choices between the following:
1. People and things (Extraversion or "E") vs Ideas and information (Introversion or "I").
2. Facts and reality (Sensing or "S") vs Possibilities and potential (Intuition or "N").
3. Logic and truth (Thinking or "T") vs Values and relationships (Feeling or "F").
4. A lifestyle that is well-structured (Judgment or "J") vs One that goes with the flow (Perception or "P"). 

The combination of these 4 preferences will determine which personality type a person would fall into. 

There are 16 in total. 